In [2]:
import psycopg2
from zipfile import ZipFile
import pandas as pd
import os
from io import StringIO
import numpy as np

zipped_folder=os.path.join('C:\\Users\\Munira\\Desktop\\Assessment','zipped_raw_data')

unzipped_folder=os.path.join('C:\\Users\\Munira\\Desktop\\Assessment','raw_data')

def connect_to_db(db_name,password):
    conn = psycopg2.connect(f"dbname={db_name} user=dataengineer host='mydb.cb7sygkkbh0u.us-east-2.rds.amazonaws.com' port=5432 password={password}")
    cur = conn.cursor()
    return conn,cur

def extract_and_unzip_files(zipped_folder, unzipped_folder):
    for zip_filename in os.listdir(zipped_folder):
        z=ZipFile(os.path.join(zipped_folder,zip_filename))
        z.extractall(unzipped_folder)

#extract_and_unzip_files(zipped_folder,unzipped_folder)

### Creating tables

In [ ]:
CREATE TABLE crash (
  Report_number VARCHAR(50) NOT NULL,
  Report_seq_no REAL NOT NULL,
  DOT_Number VARCHAR(50),
  Report_Date DATE,
  Report_State CHAR(2),
  Fatalities REAL,
  Injuries REAL,
  Tow_Away CHAR(1),
  Hazmat_released CHAR(1),
  Trafficway_Desc VARCHAR(255),
  Access_Control_Desc VARCHAR(255),
  Road_surface_Condition_Desc VARCHAR(255),
  Weather_Condition_Desc VARCHAR(255),
  Light_Condition_Desc VARCHAR(255),
  Vehicle_ID_Number VARCHAR(50),
  Vehicle_License_number VARCHAR(50),
  Vehicle_license_state CHAR(2),
  Severity_Weight REAL,
  Time_weight REAL,
  citation_issued_desc VARCHAR(255),
  seq_num REAL,
  Not_Preventable CHAR(1)
);


### Copying data into postgres

In [5]:
low_cols=[x.lower() for x in list(df.columns)]

In [6]:
def insert_csv(f, table, columns):
    connection,cursor=connect_to_db('staging','aculocity')
    try:
        cursor.copy_from(f, table, sep=',', columns=columns)
        connection.commit()
        return True
    except (psycopg2.Error) as e:
        print(e)
        return False
    finally:
        cursor.close()
        connection.close()
        


In [7]:
filepath=os.path.join(unzipped_folder,'2022Aug_Crash.csv')

In [8]:
with open(filepath) as file:
    insert_csv(file, "crash", low_cols)

invalid input syntax for type real: "REPORT_SEQ_NO"
CONTEXT:  COPY crash, line 1, column report_seq_no: "REPORT_SEQ_NO"



## Modifying raw data files

In [3]:
df=pd.read_csv(os.path.join(unzipped_folder,'2022Aug_Crash.csv'))

### Unique columns

In [545]:
df['REPORT_NUMBER'].nunique()==df.shape[0]

False

In [546]:
df.shape[0]

280498

In [547]:
df['REPORT_NUMBER'].nunique()

262448

In [549]:
df[df.duplicated(subset='REPORT_NUMBER',keep=False)].sort_values('REPORT_NUMBER')

,REPORT_NUMBER,REPORT_SEQ_NO,DOT_NUMBER,REPORT_DATE,REPORT_STATE,FATALITIES,INJURIES,TOW_AWAY,HAZMAT_RELEASED,TRAFFICWAY_DESC,...,WEATHER_CONDITION_DESC,LIGHT_CONDITION_DESC,VEHICLE_ID_NUMBER,VEHICLE_LICENSE_NUMBER,VEHICLE_LICENSE_STATE,SEVERITY_WEIGHT,TIME_WEIGHT,CITATION_ISSUED_DESC,SEQ_NUM,NOT_PREVENTABLE
7,AK0021008848,1.0,3220631.0,23-MAR-21,AK,0.0,0.0,Y,NaN,Two-Way Trafficway Divided Positive Barrier,...,No Adverse Conditions,Daylight,2NP2HM6X1KM632064,KEU996,AK,1.0,1.0,YES,1.0,NaN
8,AK0021008848,2.0,125134.0,23-MAR-21,AK,0.0,0.0,Y,NaN,Two-Way Trafficway Divided Positive Barrier,...,No Adverse Conditions,Daylight,1XKZD79X4JJ216518,JKP901,AK,1.0,1.0,UNKNOWN,1.0,Y
30,AL0000683608,1.0,2309975.0,27-AUG-20,AL,0.0,0.0,Y,N,Two-Way Trafficway Divided Positive Barrier,...,No Adverse Conditions,Daylight,1XKWDB8X87J152550,4309,NC,1.0,1.0,NO,1.0,NaN
31,AL0000683608,2.0,271424.0,27-AUG-20,AL,0.0,0.0,Y,N,Two-Way Trafficway Divided Positive Barrier,...,No Adverse Conditions,Daylight,3HSDZAPRXKN392679,1128277,AL,1.0,1.0,NO,1.0,NaN
46,AL0000684438,1.0,3412525.0,29-AUG-20,AL,0.0,3.0,N,N,Two-Way Trafficway Not Divided,...,No Adverse Conditions,Daylight,3AKJHHDR2MSME8323,1068338,IL,2.0,1.0,NO,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
280477,WY0202208859,2.0,1765424.0,15-AUG-22,WY,0.0,0.0,Y,NaN,Two-Way Trafficway Not Divided,...,No Adverse Conditions,Daylight,3HTJLTKT0GN743914,17C2554,WY,1.0,3.0,NO,1.0,NaN
280481,WY0202208919,1.0,551221.0,17-JUN-22,WY,3.0,0.0,Y,NaN,Two-Way Trafficway Divided Positive Barrier,...,Rain,Dark - Not Lighted,3HSDZTZR9NN077403,P1134133,IL,2.0,3.0,NO,1.0,NaN
280482,WY0202208919,2.0,74432.0,17-JUN-22,WY,3.0,0.0,Y,NaN,Two-Way Trafficway Divided Positive Barrier,...,Rain,Dark - Not Lighted,1XKYD49X6MJ432160,2913426,IN,2.0,3.0,NO,1.0,NaN
280488,WY0202209041,1.0,3623103.0,16-JUL-22,WY,0.0,1.0,Y,NaN,Two-Way Trafficway Divided Positive Barrier,...,No Adverse Conditions,Daylight,1FUJGLD5XGLGR1498,YP45754,CA,2.0,3.0,NO,1.0,NaN


#### Removing rows where all values are null/missing

In [510]:
df.shape

(281285, 22)

In [528]:
df.shape[0] - df.dropna().shape[0]

787

In [529]:
df=df.dropna()

### Float columns - dealing with nulls showing up as empty strings

In [530]:
df.loc[df['REPORT_SEQ_NO']=='','REPORT_SEQ_NO']=np.nan

In [531]:
df.loc[df['REPORT_SEQ_NO'].isnull(),'REPORT_SEQ_NO']=np.nan

#### Text columns - filling in nulls with empty strings

In [534]:
cols=[col for col in df.columns if df[col].dtype=='O']

In [535]:
cols

['REPORT_NUMBER',
 'REPORT_DATE',
 'REPORT_STATE',
 'TOW_AWAY',
 'HAZMAT_RELEASED',
 'TRAFFICWAY_DESC',
 'ACCESS_CONTROL_DESC',
 'ROAD_SURFACE_CONDITION_DESC',
 'WEATHER_CONDITION_DESC',
 'LIGHT_CONDITION_DESC',
 'VEHICLE_ID_NUMBER',
 'VEHICLE_LICENSE_NUMBER',
 'VEHICLE_LICENSE_STATE',
 'CITATION_ISSUED_DESC',
 'NOT_PREVENTABLE']

In [536]:
for col in cols:
    
    df[col]=df[col].fillna('')

C:\Users\Munira\AppData\Local\Temp\ipykernel_3420\1911975763.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col]=df[col].fillna('')


In [537]:
df

,REPORT_NUMBER,REPORT_SEQ_NO,DOT_NUMBER,REPORT_DATE,REPORT_STATE,FATALITIES,INJURIES,TOW_AWAY,HAZMAT_RELEASED,TRAFFICWAY_DESC,...,WEATHER_CONDITION_DESC,LIGHT_CONDITION_DESC,VEHICLE_ID_NUMBER,VEHICLE_LICENSE_NUMBER,VEHICLE_LICENSE_STATE,SEVERITY_WEIGHT,TIME_WEIGHT,CITATION_ISSUED_DESC,SEQ_NUM,NOT_PREVENTABLE
0,AK0020039340,1.0,8294.0,15-DEC-20,AK,0.0,0.0,Y,N,One-Way Trafficway Not Divided,...,Snow,Dark - Lighted,2XKHDJ9X1GM481794,GXX560,AK,1.0,1.0,UNKNOWN,1.0,
1,AK0020039542,1.0,190356.0,17-DEC-20,AK,0.0,0.0,Y,,One-Way Trafficway Not Divided,...,No Adverse Conditions,Daylight,1XKZD49X4KJ303397,KCX725,AK,1.0,1.0,YES,1.0,
2,AK0020039828,1.0,3120623.0,19-DEC-20,AK,0.0,1.0,Y,,Two-Way Trafficway Divided Positive Barrier,...,Blowing Sand Soil Dirt Or Snow,Dark - Lighted,1HTMMMML5JH568106,TOWAK,AK,2.0,1.0,UNKNOWN,1.0,
3,AK0020081042,1.0,1640646.0,27-OCT-20,AK,1.0,1.0,Y,,Two-Way Trafficway Not Divided,...,Blowing Sand Soil Dirt Or Snow,Dark - Not Lighted,2NP3LJ0X9EM239931,GSA386,AK,2.0,1.0,UNKNOWN,1.0,
4,AK0020095165,1.0,248628.0,08-DEC-20,AK,1.0,3.0,Y,,Two-Way Trafficway Not Divided,...,No Adverse Conditions,Daylight,J8DC4W16187003451,FPB425,AK,2.0,1.0,UNKNOWN,1.0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
281280,WY0202209128,1.0,3749576.0,21-AUG-22,WY,0.0,0.0,Y,,Two-Way Trafficway Divided Positive Barrier,...,No Adverse Conditions,Daylight,1XKYD49X3NJ101005,YP57687,CA,1.0,3.0,NO,1.0,
281281,WY0202209132,1.0,53467.0,09-AUG-22,WY,0.0,0.0,Y,,Two-Way Trafficway Divided Positive Barrier,...,No Adverse Conditions,Dark - Not Lighted,3HSDZAPR8MN683541,W33995,NE,1.0,3.0,NO,1.0,
281282,WY0202209156,1.0,107540.0,16-AUG-22,WY,0.0,0.0,Y,,Two-Way Trafficway Divided Unprotected Median,...,No Adverse Conditions,Daylight,1NPTL40X8DD200284,2SG524,OK,1.0,3.0,NO,1.0,
281283,WY0502207230,1.0,524401.0,10-JUL-22,WY,0.0,0.0,Y,,Two-Way Trafficway Not Divided,...,No Adverse Conditions,Dark - Not Lighted,WCDPF1CD0KT00324,KAF173,ID,1.0,3.0,NO,1.0,


In [539]:
df.to_csv(os.path.join(unzipped_folder,'2022Aug_Crash.csv'),index=False)

## Copying into postgres

In [ ]:
\copy crash FROM 'C:\\Users\Munira\Desktop\Assessment\raw_data\2022Aug_Crash.csv' DELIMITER ','
HEADER;